<a href="https://colab.research.google.com/github/murthylab/sleap/blob/main/docs/notebooks/Training_and_inference_using_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll install SLEAP, import training data into Colab using [Google Drive](https://www.google.com/drive), and run training and inference.

# Install SLEAP
Note: Before installing SLEAP check [SLEAP releases](https://github.com/murthylab/sleap/releases) page for the latest version.

In [ ]:
!pip install sleap==1.1.1

# Impoprt training data into Colab with Google Drive
We'll first prepare and export the training data from SLEAP, then upload it to Google Drive, and then mount Google Drive  into this Colab notebook.

## Create and export the training job package
A self-contained **training job package** contains a .slp file with labeled data as well as the labeled images which will be used for training. One advantage to exporting a training job package is that it doesn't depend on paths to other files (i.e., videos) to be messed up when you copy your project to another volume.

See [this guide](https://sleap.ai/guides/training-job-package.html) for exporting a training job package from SLEAP.

## Upload training job package to Google Drive
To be consistent with the examples in this notebook, name the SLEAP project `colab` and create a directory called `sleap` in the root of your Google Drive. Then upload the exported training job package `colab.slp.training_job.zip` into `sleap` directory.

If you place your training pckage somewhere else, or name it differently, adjust the paths/filenames/parameters below accordingly.

## Mount your Google Drive
Mounting your Google Drive will allow you to accessed the training job package from Colab. When prompted to log into your Google account, give Colab access, and the copy the authorization code into a field below (+ hit 'return').

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Let's set your current working directory to the directory with your training job package and unpack it there. This will ensure that the output from training (i.e., the models) and from interence (i.e., predictions) will all be saved in this directory.

In [ ]:
import os
os.chdir("/content/drive/My Drive/sleap-data")
!unzip colab.slp.training_job.zip
!ls

# Train a model

Using the pre-baked script, we'll train a model with the training profile you have exported from SLEAP.

When you start training, you'll first see the training parameters and then the training and validation loss for each training epoch.

As soon as you're satisfied with the validation loss you see for an epoch during training, you're welcome to stop training by clicking the stop button. The version of the model with the lowest validation loss is saved during training, and that's what will be used for inference.

If you don't stop training, it will run for 200 epochs or until validation loss fails to improve for some number of epochs (controlled by the early_stopping fields in the training profile).

In [ ]:
!source train-script.sh

Once training finishes, there will be a directory for each model containing all the files SLEAP needs to use this model.

Down below we'll use the model files to run inference within this notebook. But you can also download the training models for running inference from the SLEAP GUI on your computer (or anywhere else).

# Inference

Let's first upload the video we want to run inference on and name it `colab_demo.mp4`. (If your video is not named `colab_demo.mp4`, adjust the names below accordingly.)

For this demo we'll just get predictions for the first 200 frames (or you can adjust the --frames parameter below or remove it to run on the whole video).

Note: Here we assume you've trained a bottom-up model; see the end of the notebook for how to run inference with the pair of top-down models instead.

In [ ]:
!sleap-track colab_demo.mp4 \
    --frames 0-200 \
    --tracking.tracker simple \
    -m colab_demo.bottomup

When inference is finished, it will save the predictions in a file which can be opened in the GUI as a SLEAP project file. The file will be in the same directory as the video and the filename will be `{video filename}.predictions.slp`.

Let's inspect the predictions file:

In [ ]:
!sleap-inspect colab_demo.mp4.predictions.slp

You can copy this file from your Google Drive to a local drive and open it in the SLEAP GUI app (or open it directly if you have your Google Drive mounted on your local machine). If the video is in the same directory as the predictions file, SLEAP will automatically find it; otherwise, you'll be prompted to locate the video (since the path to the video on your local machine will be different than the path to the video on Colab).

## Inference with top-down models

If you trained the pair of models needed for top-down inference, you can call `sleap-track` with `-m path/to/model` for each model, like so:

In [ ]:
!sleap-track colab_demo.mp4 \
    --frames 0-200 \
    --tracking.tracker simple \
    -m colab_demo.topdown_confmaps \
    -m colab_demo.centroid